In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.cm as cm
import seaborn as sns
from matplotlib import gridspec
import matplotlib.animation as manimation
import matplotlib.patches as patches
import time
from collections import Counter
import sklearn.cluster as cluster
import time
import hdbscan
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
import re

#import plotly. as py
#import plotly.graph_objs as go
font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 16}

plt.rc('font', **font)
plt.rc('lines',lw=2) 
sns.set(font_scale = 2)

In [2]:
# dereks function for optimal leaf clustering
def _optimal_order(data, **kwargs):
    """ Optimal leaf ordering
        **kwargs passed to pdist e.g. metric='correlation'
    """
    d = pdist(data, **kwargs)
    link = linkage(d, method='average')
    optimal_order = optimal_leaf_ordering(link, d)
    return optimal_order

from scipy.spatial.distance import pdist, squareform
from fastcluster import linkage
from polo import optimal_leaf_ordering

In [3]:
KEGGmoddf = pd.read_pickle('KEGG_modules_df')
alt4 = pd.read_pickle('ObsCom_alt4')
alt = pd.read_pickle('ObsCom_alt')
alt5 = pd.read_pickle('ObsCom_alt5')
alt6 = pd.read_pickle('ObsCom_alt6')
indexswitcher = pd.read_pickle('ObsCom_indexswitcher')
altlength = pd.read_pickle('ObsCom_altlength')
clusterRes_df = pd.read_pickle('ObsCom_clusterResDF')

alt2 = pd.read_pickle('ObsCom_alt2')
alt3 = pd.read_pickle('ObsCom_alt3')
altlength = pd.read_pickle('ObsCom_altlength')
altlength.groupby('sample').sum()['chipcount']

mod_df = pd.read_pickle('KEGG_module_list')
known_genomes = pd.read_pickle('known_genomes_KEGG_module_database')
database4 = pd.read_pickle('KEGG_organism_list')
mod_df['Module_combined'] = mod_df['Module']+', '+mod_df['Module_long']
known_genomes_long = known_genomes.copy()
known_genomes_long.index = mod_df['Module_combined']

dataC = pd.read_pickle('all_genomes_dataC')
dataK = pd.read_pickle('all_genomes_dataK')
idx = list(open('idxfile.txt'))
idxcl = list(open('idxclfile.txt'))
idx = [int(i[:-1]) for i in idx]
idxcl = [int(i[:-1]) for i in idxcl]

Mmap_all_NU = pd.read_pickle('Mmap_all_NU')
Mmap_cons_NU = pd.read_pickle('Mmap_cons_NU')
Mmap_complete_NU = pd.read_pickle('Mmap_complete_NU')
Mmap_all_NU_col = pd.read_pickle('Mmap_all_NU_col')
Mmap_cons_NU_col = pd.read_pickle('Mmap_cons_NU_col')
Mmap_complete_NU_col = pd.read_pickle('Mmap_complete_NU_col')

Mmap_all_NU_grp = pd.read_pickle('Mmap_all_NU_grp')
Mmap_cons_NU_grp = pd.read_pickle('Mmap_cons_NU_grp')
Mmap_complete_NU_grp = pd.read_pickle('Mmap_complete_NU_grp')
Mmap_all_NU_grp_col = pd.read_pickle('Mmap_all_NU_grp_col')
Mmap_cons_NU_grp_col = pd.read_pickle('Mmap_cons_NU_grp_col')
Mmap_complete_NU_grp_col = pd.read_pickle('Mmap_complete_NU_grp_col')

known_genomes_all = pd.read_pickle('known_genomes_all')
known_genomes_all_grp = pd.read_pickle('known_genomes_all_grp')
known_genomes_cons = pd.read_pickle('known_genomes_cons')
known_genomes_cons_grp = pd.read_pickle('known_genomes_cons_grp')
known_genomes_complete = pd.read_pickle('known_genomes_complete')
known_genomes_complete_grp = pd.read_pickle('known_genomes_complete_grp') 

known_genomes_all_col = pd.read_pickle('known_genomes_all_col')
known_genomes_all_grp_col = pd.read_pickle('known_genomes_all_grp_col')
known_genomes_cons_col = pd.read_pickle('known_genomes_cons_col')
known_genomes_cons_grp_col = pd.read_pickle('known_genomes_cons_grp_col')
known_genomes_complete_col = pd.read_pickle('known_genomes_complete_col')
known_genomes_complete_grp_col = pd.read_pickle('known_genomes_complete_grp_col') 

# assign length to each grouped cluster, as well as ungrouped 
alt5['length'] = altlength.groupby('groupnum').sum()['length']
alt5.loc[altlength[altlength.index.isin(alt5.index)].index,'length'] = altlength[altlength.index.isin(alt5.index)]['length']

In [4]:
alt5[alt5['phylum'].str.contains('Micr')]
#alt5[alt5['clustsum'].str.contains('Obs2_meta_16')]
alt5.loc['M51']
#alt5.loc['Obs5_mini_8']
#alt5

domain                          Viruses
phylum                    dsDNA viruses
species                      Unassigned
assembly                       megameta
orig doms                VirusesViruses
orig phyl    dsDNA virusesdsDNA viruses
clustsum       Obs4_mega_11Obs4_meta_17
sample                             Obs4
length                      1.01041e+06
Name: M51, dtype: object

In [36]:
#add superclusters - superclusters generally are similar clusters over different experiments

#alt['SuperCluster']=''
#alt[alt['groupnum'].isin(['M59','M34','M00'])]['SuperCluster']=1
"""alt.loc[alt[alt['groupnum'].isin(['M59','M34','M00'])].index,'SuperCluster'] = 1
alt.loc[alt[alt['idx'].isin(['Obs4_meta_8',
                             'Obs6_meta_18','Obs2_mega_3'])].index,'SuperCluster'] = 2
alt.loc[alt[alt['idx'].isin(['Obs6_meta_23'])].index,'SuperCluster'] = 3
alt.loc[alt[alt['groupnum'].isin(['M50','M04','M26'])].index,'SuperCluster'] = 3
alt.loc[alt[alt['groupnum'].isin(['M56','M75'])].index,'SuperCluster'] = 4
alt.loc[alt[alt['groupnum'].isin(['M35'])].index,'SuperCluster'] = 5
alt.loc[alt[alt['idx'].isin(['Obs5_mini_8'])].index,'SuperCluster'] = 5"""
# these have a similar set of drug resistance genes
alt[alt['groupnum'].isin(['M58','M60','M31','M19','M39'])]
alt[alt['idx'].isin(['Obs5_mini_18'])]
#unique functionality 
alt[alt['groupnum']=='M30']
alt[alt['groupnum']=='M29']

# save to supercluster 
alt.to_pickle('ObsCom_alt')

In [42]:
database_gen_combined = database4.set_index('code')[['kingdom','phylum']]
database_gen_combined.columns = ['domain','phylum']
altt = alt[['dominant domain','dominant phylum']]
altt.columns = ['domain','phylum']
database_gen_combined = database_gen_combined.append(altt[['domain','phylum']])

database_gen_combined_grp = database4.set_index('code')[['kingdom','phylum']]
database_gen_combined_grp.columns = ['domain','phylum']
database_gen_combined_grp = database_gen_combined.append(alt5[['domain','phylum']])

In [49]:
############################################################################################
### plot the overview plots with all KEGG hits and all genomes 
############################################################################################

def plotClusterOverviewTot_gen(KEGGhitType,group,phylum):
    
    if group != "YES":
        
        if KEGGhitType == 'all':
            dataframe = pd.read_pickle('known_genomes_all')
            dataframe_col = pd.read_pickle('known_genomes_all_col')
            subtitle = 'all KEGG Module hits'
        elif KEGGhitType == 'conservative':
            dataframe = pd.read_pickle('known_genomes_cons')
            dataframe_col = pd.read_pickle('known_genomes_cons_col')
            subtitle = 'complete KEGG Modules and Modules missing one block'
        elif KEGGhitType == 'complete':
            dataframe = pd.read_pickle('known_genomes_complete')
            dataframe_col = pd.read_pickle('known_genomes_complete_col')
            subtitle = 'complete KEGG Modules only'
        else:
            import sys
            sys.exit("aborted, enter correct df type")
        dataframe = dataframe.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt[alt['dominant phylum'].str.contains(phylum)].index)]
        dataframe_col = dataframe_col.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt[alt['dominant phylum'].str.contains(phylum)].index)]
        alt55 = database_gen_combined.loc[dataframe.columns,:]
        grouped = ', ungrouped clusters,\n'
    else:
        
        if KEGGhitType == 'all':
            dataframe = pd.read_pickle('known_genomes_all_grp')
            dataframe_col = pd.read_pickle('known_genomes_all_grp_col')
            subtitle = 'all KEGG Module hits'
        elif KEGGhitType == 'conservative':
            dataframe = pd.read_pickle('known_genomes_cons_grp')
            dataframe_col = pd.read_pickle('known_genomes_cons_grp_col')
            subtitle = 'complete KEGG Modules and Modules missing one block'
        elif KEGGhitType == 'complete':
            dataframe = pd.read_pickle('known_genomes_complete_grp')
            dataframe_col = pd.read_pickle('known_genomes_complete_grp_col')
            subtitle = 'complete KEGG Modules only'
        else:
            import sys
            sys.exit("aborted, enter correct df type")
        dataframe = dataframe.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt5[alt5['phylum'].str.contains(phylum)].index)]
        dataframe_col = dataframe_col.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt5[alt5['phylum'].str.contains(phylum)].index)]
        alt55 = database_gen_combined_grp.loc[dataframe.columns,:]
        grouped = ', grouped clusters,\n'
    
    """
    0 = Energy metabolism
    1 = Carb and lipid metabolism
    2 = Nucleic acid and aa metabolism
    3 = Secondary metabolism
    4 = Genetic info proc.
    5 = Env. info proc.
    6 = Metabolism
    7 = Cellular processes
    8 = Gene set
    """
    
    completeness = KEGGhitType
    

    ############################################################################################
    # cluster correlation for KEGG B-level = environmental info processing
    ############################################################################################

    dataC = dataframe.fillna(0).corr().dropna(how='all').T.dropna(how='all')
    clustindexlist = dataC.index
    
    D = pdist(dataC, 'euclidean')
    Z = linkage(D, 'ward')
    row_link = _optimal_order(dataC, metric='correlation')

    optimal_Z = optimal_leaf_ordering(Z, D)

    cgClust = sns.clustermap(dataC, row_linkage=row_link, col_linkage=row_link,figsize=(20,40))


    ############################################################################################
    # KEGG-clust correlation for KEGG B-level = environmental info processing
    ############################################################################################

    #dataK = alt6.ix[idxblevdf['column'].values,:].T.corr() #np.random.choice(10000, (n, 1), replace=False)
    dataK = dataframe.fillna(0).T.corr()

    dataK = dataK.dropna(how='all').T.dropna(how='all')

    D = pdist(dataK, 'euclidean')
    Z = linkage(D, 'ward')
    row_link = _optimal_order(dataK, metric='correlation')

    optimal_Z = optimal_leaf_ordering(Z, D)

    cgKEGG_blevel = sns.clustermap(dataK, row_linkage=row_link, col_linkage=row_link,figsize=(10,10))

    # sort by clustermap, 
    for blev in range(0,1):

        lut = {'Archaea':'r','Bacteria':'b','Unassigned':'k','Eukaryota':'c','Viruses':'m'}

        
            
        idx = cgKEGG_blevel.dendrogram_col.reordered_ind
        idxcl = cgClust.dendrogram_col.reordered_ind
            
        
        cmap=ListedColormap(["#e74c3c", "#3498db", "#2ecc71", "#95a5a6", "#34495e","#9b59b6"])
        cmap=ListedColormap(["#e74c3c", "#9b59b6", "#2ecc71", "#95a5a6", "#34495e","#3498db"])#[red,lightblue,green,GREY,darkblue,purple]
        #cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))
        
        

        
        
        f = plt.figure()
        gs = gridspec.GridSpec(45,38)


        dataK2 = dataK.copy()
        dataK2.index = mod_df.set_index('Module').loc[dataK.index]['Module_combined']

        ax1 = f.add_subplot(gs[4:36,0:11])
        sns.heatmap(dataK2.iloc[idx,idx],cbar=False,ax=ax1,xticklabels=True,yticklabels=True,cmap="RdBu_r",vmin=-1,vmax=1)
        plt.xticks([]);plt.yticks(fontsize=35)
        plt.xlabel('KEGG modules',fontsize=30,labelpad=40)

        ###############################################
        ######### the sorter #####################
        ###############################################
        #idx2 = alt.sort_values('assembly').index
        ###############################################
        ###############################################


        ax2 = f.add_subplot(gs[4:36,13:32])
        sns.heatmap(dataframe_col.fillna(0).loc[dataK.iloc[idx,:].index,dataC.iloc[idxcl,:].index]
                    ,ax=ax2,cbar=False,linewidth=0.5,cmap=cmap,vmin=-1,vmax=2,xticklabels=True)
        plt.yticks([])
        plt. title(phylum+grouped+subtitle,fontsize=70,y=1.05)
        #plt.xlabel('Cluster number')
        #plt.xticks(rotation=0)

        ax13 = f.add_subplot(gs[37:,13:32])
        sns.heatmap(dataC.iloc[idxcl,idxcl],cbar=False,ax=ax13,cmap="RdBu_r",vmin=-1,vmax=1)
        plt.yticks([])
        plt.xticks([])

        ##############################################################################
        #################### plot hbars on right #####################################
        ##############################################################################
        

        database = dataframe.loc[dataK.iloc[idx,:].index,alt55[~alt55.index.str.contains('Obs')].index]
        database = database.loc[:,alt55[~alt55.index.str.contains('M')].index].T.sum().fillna(0)
        experim = dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55.index.str.contains('M')].index]
        experim = experim.join(dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55.index.str.contains('Obs')].index],how='outer').T.sum()
        #vir = dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55['domain']=='Viruses'].index].T.sum()
        #una = dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55['domain']=='Unassigned'].index].T.sum()

        ax3 = f.add_subplot(gs[4:36,32]) #genomes from KEGG database
        database = database/database.sum()
        database.plot.barh(ax=ax3,sharey=True, color='#34495e')
        ax3.grid(False)
        plt.title('Database',fontsize=25)
        plt.xticks(rotation=90)
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()

        ax4 = f.add_subplot(gs[4:36,33]) #found modules in experiment 
        experim = experim/experim.sum()
        experim.plot.barh(ax=ax4,sharey=True, color="#3498db")
        ax4.grid(False)
        plt.title('Experiment',fontsize=25)
        plt.xticks(rotation=90)
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()

        """
        ax5 = f.add_subplot(gs[4:36,34]) #Viruses
        vir = vir/vir.sum()
        vir.plot.barh(ax=ax5,sharey=True, color="#9b59b6")
        ax5.grid(False)
        plt.title('Viruses',fontsize=25)
        plt.xticks(rotation=90)
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()"""

        """
        ax6 = f.add_subplot(gs[4:36,35]) #Unassigned
        una = una/una.sum()
        una.plot.barh(ax=ax6,sharey=True, color="#3498db")
        ax6.grid(False)
        plt.title('Unassigned',fontsize=25)
        plt.xticks(rotation=90)
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()"""

        ax14 = f.add_subplot(gs[4:36,34:36]) #Archaea-Bacteria    
        database = database/database.max()
        experim = experim/experim.max()
        diff = experim-database

        diff.plot.barh(ax=ax14,sharey=True, color="#e79f3c")
        ax14.grid(False)
        plt.title('Expt-Datab\ndifference',fontsize=25)
        plt.xticks(rotation=90)
        plt.axvline(x=0);plt.xlim(-1,1)
        plt.gca().invert_yaxis()

        ##############################################################################
        ##########################   plot top bars   #################################
        ##############################################################################

        #ax7 = f.add_subplot(gs[0,13:32]) #phylum
        
        """
        col = sns.color_palette("cubehelix", len(list(alt55.loc[dataC.index]['phylum'].unique())))
        keys = list(alt55.loc[dataC.index]['phylum'].unique());values = [i for i in col]
        lut = dict(zip(keys,values))
        lut.update({'Unassigned':"#3498db"})
        color = alt55['phylum'].map(lut)
        print(color)
        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                   ncol=6, mode="expand", borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax7.set_ylabel('Phylum', rotation=0, fontsize=30, labelpad=90)



        #3498db
        ax8 = f.add_subplot(gs[1,13:32]) #domain
        lut = {'Archaea':"#e74c3c",'Bacteria':'#34495e','Unassigned':"#3498db",'Eukaryota':'c','Viruses':"#9b59b6"}

        color = alt55['domain'].map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=2, borderaxespad=0.,fontsize=20)
        
        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax8.set_ylabel('Kingdom', rotation=0, fontsize=30, labelpad=90)

        """

        """        
        ax9 = f.add_subplot(gs[2,13:32]) #assembly type 
        

            col = sns.color_palette("BrBG", len(alt5['assembly'].unique()))
            keys = np.sort(alt5['assembly'].unique())
            keys = keys[[1,2,0,5,6,4,3]] # order from most bulk to most minimetagenomic assembly type (with mega being the lesser of two bulks)
            values = [i for i in col]
            lut = dict(zip(keys,values))
            color = alt5['assembly'].map(lut)
        else:
            col = sns.color_palette("BrBG", len(alt['assembly'].unique()))
            keys = np.sort(alt['assembly'].unique())
            values = [i for i in col]
            lut = dict(zip(keys,values))
            color = alt['assembly'].map(lut)
                
        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=3, borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        plt.xticks([]);plt.yticks([])
        ax9.set_ylabel('Assembly', rotation=0, fontsize=30, labelpad=90)




        ax10 = f.add_subplot(gs[3,13:32]) #experiment
        col = sns.color_palette("Set2", 5)
        if group == 'YES':
            lut = {'Obs2':col[0],'Obs3':col[1],'Obs4':col[2],'Obs5':col[3],'Obs6':col[4]}
            color2 = alt5['sample'].map(lut)
        else:
            lut = {'Obsidian2':col[0],'Obsidian3':col[1],'Obsidian4':col[2],'Obsidian5':col[3],'Obsidian6':col[4]}
            color2 = alt['sample'].map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=3, borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color2[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax10.set_ylabel('Experiment', rotation=0, fontsize=30, labelpad=100)
        """

        ##############################################################################
        ##########################   plot left side bars  ############################
        ##############################################################################
        Blevel = blev
        
        """
        0 = Energy metabolism
        1 = Carb and lipid metabolism
        2 = Nucleic acid and aa metabolism
        3 = Secondary metabolism
        4 = Genetic info proc.
        5 = Env. info proc.
        6 = Metabolism
        7 = Cellular processes
        8 = Gene set"""
        
        # B-level assignment
        ax11 = f.add_subplot(gs[4:36,11])
        col = sns.color_palette("gist_earth", len(KEGGmoddf.B.unique())-1)
        keys = list(KEGGmoddf.B.unique());keys = keys[0:-1];values = [i for i in col]
        keysC = KEGGmoddf[KEGGmoddf['B']==keys[Blevel]].C.unique();

        lut = dict(zip(keys,values))
        idxlist = dataK.iloc[idx,:].index
        #idxlist2 = [i[0:6] for i in idxlist]
        D_group = KEGGmoddf.groupby('D-module').sum()

        for i in D_group.index:
            a = D_group.loc[i,'B']
            for j in keys:
                if a.find(j)!=-1:
                    D_group.loc[i,'B'] = j
            c = D_group.loc[i,'C']
            for j in KEGGmoddf.C.unique():
                if c.find(j)!=-1:
                    D_group.loc[i,'C'] = j

        B_list  = D_group.loc[idxlist]['B']
        #B_list.replace(B_list.index,idxlist)
        color = B_list.map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(-12, 1.02, 1., .102), loc=3,
                   ncol=1, mode="expand", borderaxespad=0.,fontsize=30)
        
        plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
        plt.xticks([]);plt.yticks([])
        plt.ylim(-.5,len(dataK)-.5)
        plt.xlim(0,1)
        ax11.set_xlabel('B', rotation=0, fontsize=30, labelpad=40)
        plt.gca().invert_yaxis()


        #C-level assignment

        ax12 = f.add_subplot(gs[4:36,12])
        col = sns.color_palette("Set3", len(keysC));values = [i for i in col]
        lutC = dict(zip(keysC,values))

        C_list  = D_group.loc[idxlist]['C']
        C_list.replace(C_list.index,idxlist)
        color = C_list.map(lutC)
        color = color.fillna('None')

        plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
        plt.xticks([]);plt.yticks([])
        plt.ylim(-.5,len(dataK)-.5)
        plt.xlim(0,1)
        ax12.set_xlabel('C', rotation=0, fontsize=30, labelpad=40)
        plt.gca().invert_yaxis()

        for x,y in lutC.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(-8, 1.02, 1., .102), loc=3,
                   ncol=1, mode="expand", borderaxespad=0.,fontsize=30)

        
        
        gs.update(wspace=.05,hspace=.05)
        
        f.set_figheight(55)
        #plt.show()
        if group =='YES':
            f.set_figwidth(60)
            f.savefig('OverviewHeatmaps/Grouped/Known/Obsidian_grouped_KEGG_'+completeness+'_'+phylum+'_'+keys[Blevel])
            #f.savefig('OverviewHeatmaps/Grouped/Known/pdfs/Obsidian_grouped_KEGG_'+completeness+'_'+keys[Blevel]+'.pdf')
        else:
            f.set_figwidth(45)
            f.savefig('OverviewHeatmaps/Ungrouped/Known/Obsidian_ungrouped_KEGG_'+completeness+'_'+phylum+'_'+keys[Blevel])
            #f.savefig('OverviewHeatmaps/Ungrouped/Known/pdfs/Obsidian_ungrouped_KEGG_'+completeness+'_'+keys[Blevel]+'.pdf')


In [52]:
plotClusterOverviewTot_gen('conservative','YES','Nitrospirae')


In [56]:
an = (1.5e-1*0.7e-2)/3e9
np.sqrt(an)

5.9160797830996155e-07

In [12]:
database4[database4['domain']=='Prokaryotes']['phylum'].unique()

"""for p in phylumlist:
    phylum = p;#'Euryarchaeota'
    plotClusterOverviewTot_gen('all','YES',phylum)
    plotClusterOverviewTot_gen('complete','YES',phylum)
    plotClusterOverviewTot_gen('conservative','YES',phylum)
    plotClusterOverviewTot_gen('all','nope',phylum)
    plotClusterOverviewTot_gen('complete','nope',phylum)
    plotClusterOverviewTot_gen('conservative','nope',phylum)"""

array(['Gammaproteobacteria - Enterobacteria',
       'Gammaproteobacteria - Others', 'Betaproteobacteria',
       'Epsilonproteobacteria', 'Deltaproteobacteria',
       'Alphaproteobacteria', 'Other proteobacteria',
       'Firmicutes - Bacilli', 'Firmicutes - Clostridia',
       'Firmicutes - Others', 'Tenericutes', 'Actinobacteria',
       'Cyanobacteria', 'Melainabacteria', 'Chloroflexi',
       'Deinococcus-Thermus', 'Armatimonadetes',
       'Unclassified Terrabacteria group', 'Chlamydiae', 'Verrucomicrobia',
       'Planctomycetes', 'Kiritimatiellaeota', 'Spirochaetes',
       'Acidobacteria', 'Elusimicrobia', 'Fusobacteria', 'Synergistetes',
       'Fibrobacteres', 'Gemmatimonadetes', 'Bacteroidetes', 'Chlorobi',
       'Cloacimonetes', 'Aquificae', 'Thermotogae', 'Caldiserica',
       'Chrysiogenetes', 'Deferribacteres', 'Calditrichaeota',
       'Dictyoglomi', 'Nitrospirae', 'Thermodesulfobacteria',
       'Saccharibacteria', 'Peregrinibacteria', 'Other',
       'Unclassified

In [7]:
alt['dominant phylum'].unique()

array(['Unassigned', 'Crenarchaeota', 'Nanoarchaeota', 'Aquificae',
       'Proteobacteria', 'Thermodesulfobacteria', 'Thaumarchaeota',
       'Ignavibacteriae', 'Fervidibacteria', 'Firmicutes', 'Dictyoglomi',
       'Aigarchaeota', 'Planctomycetes', 'Chloroflexi', 'Chlamydiae',
       'Euryarchaeota', 'Bacteroidetes', 'Candidatus Bathyarchaeota',
       'dsDNA viruses', 'Fervidibacteria AND Ignavibacteriae',
       'Thermotogae', 'Cyanobacteria', 'Aminicenantes', 'Parcubacteria',
       'Nitrospirae', 'Candidatus Micrarchaeota',
       'dsDNA viruses AND dsDNA viruses, no RNA stage'], dtype=object)

In [106]:
alt5[alt5['phylum'].str.contains('Micrarchaeota')]

,domain,phylum,species,assembly,orig doms,orig phyl,clustsum,sample,length
M72,Archaea,Candidatus Micrarchaeota,unclassified,megameta,ArchaeaArchaea,Candidatus MicrarchaeotaCandidatus Micrarchaeota,Obs6_mega_12Obs6_meta_11,Obs6,3187531.0


In [10]:
phylumlist = ['Crenarchaeota', 'Thermotogae', 'Cyanobacteria', 'Nanoarchaeota', 'Aquificae',
              'Nitrospirae','Thermodesulfobacteria', 'Thaumarchaeota','Chloroflexi', 'Chlamydiae','Aminicenantes', 'Parcubacteria',]
phylumlist = 'Micrarchaeota'

In [8]:
############################################################################################
### plot the overview plots with all KEGG hits and all genomes 
############################################################################################

def plotClusterOverviewTot_gen_names(KEGGhitType,group,phylum):
    
    if group != "YES":
        
        if KEGGhitType == 'all':
            dataframe = pd.read_pickle('known_genomes_all')
            dataframe_col = pd.read_pickle('known_genomes_all_col')
            subtitle = 'all KEGG Module hits'
        elif KEGGhitType == 'conservative':
            dataframe = pd.read_pickle('known_genomes_cons')
            dataframe_col = pd.read_pickle('known_genomes_cons_col')
            subtitle = 'complete KEGG Modules and Modules missing one block'
        elif KEGGhitType == 'complete':
            dataframe = pd.read_pickle('known_genomes_complete')
            dataframe_col = pd.read_pickle('known_genomes_complete_col')
            subtitle = 'complete KEGG Modules only'
        else:
            import sys
            sys.exit("aborted, enter correct df type")
        dataframe = dataframe.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt[alt['dominant phylum'].str.contains(phylum)].index)]
        dataframe_col = dataframe_col.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt[alt['dominant phylum'].str.contains(phylum)].index)]
        alt55 = database_gen_combined.loc[dataframe.columns,:]
        grouped = ', ungrouped clusters,\n'
    else:
        
        if KEGGhitType == 'all':
            dataframe = pd.read_pickle('known_genomes_all_grp')
            dataframe_col = pd.read_pickle('known_genomes_all_grp_col')
            subtitle = 'all KEGG Module hits'
        elif KEGGhitType == 'conservative':
            dataframe = pd.read_pickle('known_genomes_cons_grp')
            dataframe_col = pd.read_pickle('known_genomes_cons_grp_col')
            subtitle = 'complete KEGG Modules and Modules missing one block'
        elif KEGGhitType == 'complete':
            dataframe = pd.read_pickle('known_genomes_complete_grp')
            dataframe_col = pd.read_pickle('known_genomes_complete_grp_col')
            subtitle = 'complete KEGG Modules only'
        else:
            import sys
            sys.exit("aborted, enter correct df type")
        dataframe = dataframe.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt5[alt5['phylum'].str.contains(phylum)].index)]
        dataframe_col = dataframe_col.loc[:,list(database4[database4['phylum'].str.contains(phylum)].set_index('code').index)+list(alt5[alt5['phylum'].str.contains(phylum)].index)]
        alt55 = database_gen_combined_grp.loc[dataframe.columns,:]
        grouped = ', grouped clusters,\n'
    
    """
    0 = Energy metabolism
    1 = Carb and lipid metabolism
    2 = Nucleic acid and aa metabolism
    3 = Secondary metabolism
    4 = Genetic info proc.
    5 = Env. info proc.
    6 = Metabolism
    7 = Cellular processes
    8 = Gene set
    """
    
    completeness = KEGGhitType
    

    ############################################################################################
    # cluster correlation for KEGG B-level = environmental info processing
    ############################################################################################

    dataC = dataframe.fillna(0).corr().dropna(how='all').T.dropna(how='all')
    clustindexlist = dataC.index
    
    D = pdist(dataC, 'euclidean')
    Z = linkage(D, 'ward')
    row_link = _optimal_order(dataC, metric='correlation')

    optimal_Z = optimal_leaf_ordering(Z, D)

    cgClust = sns.clustermap(dataC, row_linkage=row_link, col_linkage=row_link,figsize=(20,40))


    ############################################################################################
    # KEGG-clust correlation for KEGG B-level = environmental info processing
    ############################################################################################

    #dataK = alt6.ix[idxblevdf['column'].values,:].T.corr() #np.random.choice(10000, (n, 1), replace=False)
    dataK = dataframe.fillna(0).T.corr()

    dataK = dataK.dropna(how='all').T.dropna(how='all')

    D = pdist(dataK, 'euclidean')
    Z = linkage(D, 'ward')
    row_link = _optimal_order(dataK, metric='correlation')

    optimal_Z = optimal_leaf_ordering(Z, D)

    cgKEGG_blevel = sns.clustermap(dataK, row_linkage=row_link, col_linkage=row_link,figsize=(10,10))

    # sort by clustermap, 
    for blev in range(0,1):

        lut = {'Archaea':'r','Bacteria':'b','Unassigned':'k','Eukaryota':'c','Viruses':'m'}

        
            
        idx = cgKEGG_blevel.dendrogram_col.reordered_ind
        idxcl = cgClust.dendrogram_col.reordered_ind
            
        
        cmap=ListedColormap(["#e74c3c", "#3498db", "#2ecc71", "#95a5a6", "#34495e","#9b59b6"])
        cmap=ListedColormap(["#e74c3c", "#9b59b6", "#2ecc71", "#95a5a6", "#34495e","#3498db"])#[red,lightblue,green,GREY,darkblue,purple]
        #cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))
        
        

        
        
        f = plt.figure()
        gs = gridspec.GridSpec(45,38)


        dataK2 = dataK.copy()
        dataK2.index = mod_df.set_index('Module').loc[dataK.index]['Module_combined']
        dataframe_col = dataframe_col.fillna(0).loc[dataK.iloc[idx,:].index,dataC.iloc[idxcl,:].index]
        
        
        dataframe_col.index = mod_df.set_index('Module').loc[dataframe_col.index,:]['Module_combined']
        
     
        ax1 = f.add_subplot(gs[4:36,:1])
        sns.heatmap(dataK2.iloc[idx,idx],cbar=False,ax=ax1,xticklabels=True,yticklabels=True,cmap="RdBu_r",vmin=-1,vmax=1)
        plt.xticks([]);plt.yticks(fontsize=15)
        plt.xlabel('KEGG modules',fontsize=30,labelpad=40)

        ###############################################
        ######### the sorter #####################
        ###############################################
        #idx2 = alt.sort_values('assembly').index
        ###############################################
        ###############################################

        
        ax2 = f.add_subplot(gs[4:36,2:32])
        sns.heatmap(dataframe_col
                    ,ax=ax2,cbar=False,linewidth=0.5,cmap=cmap
                    ,vmin=-1,vmax=2,xticklabels=True,yticklabels=True)
        
        #plt.ylabel()
        plt.title(phylum+grouped+subtitle,fontsize=25,y=1.05)
        
        #plt.xlabel('Cluster number')
        #plt.xticks(rotation=0)

        """        ax13 = f.add_subplot(gs[37:,13:32])
        sns.heatmap(dataC.iloc[idxcl,idxcl],cbar=False,ax=ax13,cmap="RdBu_r",vmin=-1,vmax=1)
        plt.yticks([])
        plt.xticks([])"""

        ##############################################################################
        #################### plot hbars on right #####################################
        ##############################################################################
        

        database = dataframe.loc[dataK.iloc[idx,:].index,alt55[~alt55.index.str.contains('Obs')].index]
        database = database.loc[:,alt55[~alt55.index.str.contains('M')].index].T.sum().fillna(0)
        experim = dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55.index.str.contains('M')].index]
        experim = experim.join(dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55.index.str.contains('Obs')].index],how='outer').T.sum()
        #vir = dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55['domain']=='Viruses'].index].T.sum()
        #una = dataframe.loc[dataK.iloc[idx,:].index,alt55[alt55['domain']=='Unassigned'].index].T.sum()

        ax3 = f.add_subplot(gs[4:36,32]) #genomes from KEGG database
        database = database/database.sum()
        database.plot.barh(ax=ax3,sharey=True, color='#34495e')
        ax3.grid(False)
        plt.title('Database',fontsize=15,rotation=45,y=1.05,x=1.05)
        #plt.xticks(rotation=90)
        plt.xticks([])
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()

        ax4 = f.add_subplot(gs[4:36,33]) #found modules in experiment 
        experim = experim/experim.sum()
        experim.plot.barh(ax=ax4,sharey=True, color="#3498db")
        ax4.grid(False)
        plt.title('Experiment',fontsize=15,rotation=45,y=1.055,x=1.05)
        #plt.xticks(rotation=90)
        plt.xticks([])
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()

        """
        ax5 = f.add_subplot(gs[4:36,34]) #Viruses
        vir = vir/vir.sum()
        vir.plot.barh(ax=ax5,sharey=True, color="#9b59b6")
        ax5.grid(False)
        plt.title('Viruses',fontsize=25)
        plt.xticks(rotation=90)
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()"""

        """
        ax6 = f.add_subplot(gs[4:36,35]) #Unassigned
        una = una/una.sum()
        una.plot.barh(ax=ax6,sharey=True, color="#3498db")
        ax6.grid(False)
        plt.title('Unassigned',fontsize=25)
        plt.xticks(rotation=90)
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()"""

        ax14 = f.add_subplot(gs[4:36,34:36]) #Archaea-Bacteria    
        database = database/database.max()
        experim = experim/experim.max()
        diff = experim-database

        diff.plot.barh(ax=ax14,sharey=True, color="#e79f3c")
        ax14.grid(False)
        plt.title('Expt-Datab\ndifference',fontsize=15,rotation=45,y=1.05,x=1.01)
        #plt.xticks(rotation=90)
        plt.axvline(x=0);plt.xlim(-1,1)
        plt.gca().invert_yaxis()
        plt.xticks([])

        ##############################################################################
        ##########################   plot top bars   #################################
        ##############################################################################

        #ax7 = f.add_subplot(gs[0,13:32]) #phylum
        
        """
        col = sns.color_palette("cubehelix", len(list(alt55.loc[dataC.index]['phylum'].unique())))
        keys = list(alt55.loc[dataC.index]['phylum'].unique());values = [i for i in col]
        lut = dict(zip(keys,values))
        lut.update({'Unassigned':"#3498db"})
        color = alt55['phylum'].map(lut)
        print(color)
        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                   ncol=6, mode="expand", borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax7.set_ylabel('Phylum', rotation=0, fontsize=30, labelpad=90)



        #3498db
        ax8 = f.add_subplot(gs[1,13:32]) #domain
        lut = {'Archaea':"#e74c3c",'Bacteria':'#34495e','Unassigned':"#3498db",'Eukaryota':'c','Viruses':"#9b59b6"}

        color = alt55['domain'].map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=2, borderaxespad=0.,fontsize=20)
        
        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax8.set_ylabel('Kingdom', rotation=0, fontsize=30, labelpad=90)

        """

        """        
        ax9 = f.add_subplot(gs[2,13:32]) #assembly type 
        

            col = sns.color_palette("BrBG", len(alt5['assembly'].unique()))
            keys = np.sort(alt5['assembly'].unique())
            keys = keys[[1,2,0,5,6,4,3]] # order from most bulk to most minimetagenomic assembly type (with mega being the lesser of two bulks)
            values = [i for i in col]
            lut = dict(zip(keys,values))
            color = alt5['assembly'].map(lut)
        else:
            col = sns.color_palette("BrBG", len(alt['assembly'].unique()))
            keys = np.sort(alt['assembly'].unique())
            values = [i for i in col]
            lut = dict(zip(keys,values))
            color = alt['assembly'].map(lut)
                
        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=3, borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        plt.xticks([]);plt.yticks([])
        ax9.set_ylabel('Assembly', rotation=0, fontsize=30, labelpad=90)




        ax10 = f.add_subplot(gs[3,13:32]) #experiment
        col = sns.color_palette("Set2", 5)
        if group == 'YES':
            lut = {'Obs2':col[0],'Obs3':col[1],'Obs4':col[2],'Obs5':col[3],'Obs6':col[4]}
            color2 = alt5['sample'].map(lut)
        else:
            lut = {'Obsidian2':col[0],'Obsidian3':col[1],'Obsidian4':col[2],'Obsidian5':col[3],'Obsidian6':col[4]}
            color2 = alt['sample'].map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=3, borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color2[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax10.set_ylabel('Experiment', rotation=0, fontsize=30, labelpad=100)
        """

        ##############################################################################
        ##########################   plot left side bars  ############################
        ##############################################################################
        Blevel = blev
        
        """
        0 = Energy metabolism
        1 = Carb and lipid metabolism
        2 = Nucleic acid and aa metabolism
        3 = Secondary metabolism
        4 = Genetic info proc.
        5 = Env. info proc.
        6 = Metabolism
        7 = Cellular processes
        8 = Gene set"""
        
        # B-level assignment
        #ax11 = f.add_subplot(gs[4:36,1])
        col = sns.color_palette("gist_earth", len(KEGGmoddf.B.unique())-1)
        keys = list(KEGGmoddf.B.unique());keys = keys[0:-1];values = [i for i in col]
        keysC = KEGGmoddf[KEGGmoddf['B']==keys[Blevel]].C.unique();

        lut = dict(zip(keys,values))
        idxlist = dataK.iloc[idx,:].index
        #idxlist2 = [i[0:6] for i in idxlist]
        D_group = KEGGmoddf.groupby('D-module').sum()

        for i in D_group.index:
            a = D_group.loc[i,'B']
            for j in keys:
                if a.find(j)!=-1:
                    D_group.loc[i,'B'] = j
            c = D_group.loc[i,'C']
            for j in KEGGmoddf.C.unique():
                if c.find(j)!=-1:
                    D_group.loc[i,'C'] = j

        B_list  = D_group.loc[idxlist]['B']
        #B_list.replace(B_list.index,idxlist)
        color = B_list.map(lut)

        """for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(-12, 1.02, 1., .102), loc=3,
                   ncol=1, mode="expand", borderaxespad=0.,fontsize=30)"""
        
        """plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
        plt.xticks([]);plt.yticks([])
        plt.ylim(-.5,len(dataK)-.5)
        plt.xlim(0,1)
        ax11.set_xlabel('B', rotation=0, fontsize=30, labelpad=40)
        plt.gca().invert_yaxis()"""


        #C-level assignment

        ax12 = f.add_subplot(gs[4:36,1])
        col = sns.color_palette("Set3", len(keysC));values = [i for i in col]
        lutC = dict(zip(keysC,values))

        C_list  = D_group.loc[idxlist]['C']
        C_list.replace(C_list.index,idxlist)
        color = C_list.map(lutC)
        color = color.fillna('None')

        plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
        plt.xticks([]);plt.yticks([])
        plt.ylim(-.5,len(dataK)-.5)
        plt.xlim(0,1)
        #ax12.set_xlabel('C', rotation=0, fontsize=30, labelpad=40)
        plt.gca().invert_yaxis()
        
        for x,y in lutC.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(-7, 1.02, 1., .102), loc=3,
                   ncol=2, mode="expand", borderaxespad=0.,fontsize=20)
        
        
        
        gs.update(wspace=.05,hspace=.05)
        
        f.set_figheight(35)
        #plt.show()
        if group =='YES':
            f.set_figwidth(35)
            f.savefig('OverviewHeatmaps/Grouped/Known/Obsidian_grouped_name_KEGG_'+completeness+'_'+phylum+'_'+keys[Blevel])
            #f.savefig('OverviewHeatmaps/Grouped/Known/pdfs/Obsidian_grouped_KEGG_'+completeness+'_'+keys[Blevel]+'.pdf')
        else:
            f.set_figwidth(35)
            f.savefig('OverviewHeatmaps/Ungrouped/Known/Obsidian_ungrouped_name_KEGG_'+completeness+'_'+phylum+'_'+keys[Blevel])
            #f.savefig('OverviewHeatmaps/Ungrouped/Known/pdfs/Obsidian_ungrouped_KEGG_'+completeness+'_'+keys[Blevel]+'.pdf')


In [158]:
plotClusterOverviewTot_gen_names('conservative','nope',phylumlist)

/usr/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [11]:
KEGGmoddf[KEGGmoddf['D'].str.contains('Anoxygenic')]

,A,B,C,D-module,D,E-module,E
2340,Structural complex,Energy metabolism,Photosynthesis,M00597,Anoxygenic photosystem II [BR:ko00194],K08928,pufL; photosynthetic reaction center L subunit
2341,Structural complex,Energy metabolism,Photosynthesis,M00597,Anoxygenic photosystem II [BR:ko00194],K08929,pufM; photosynthetic reaction center M subunit
2342,Structural complex,Energy metabolism,Photosynthesis,M00598,Anoxygenic photosystem I [BR:ko00194],K08940,pscA; photosystem P840 reaction center large s...
2343,Structural complex,Energy metabolism,Photosynthesis,M00598,Anoxygenic photosystem I [BR:ko00194],K08941,pscB; photosystem P840 reaction center iron-su...
2344,Structural complex,Energy metabolism,Photosynthesis,M00598,Anoxygenic photosystem I [BR:ko00194],K08942,pscC; photosystem P840 reaction center cytochr...
2345,Structural complex,Energy metabolism,Photosynthesis,M00598,Anoxygenic photosystem I [BR:ko00194],K08943,pscD; photosystem P840 reaction center protein...
4735,Signature module,Gene set,Metabolic capacity,M00612,Anoxygenic photosynthesis in purple bacteria,M00597,Anoxygenic photosystem II
4736,Signature module,Gene set,Metabolic capacity,M00612,Anoxygenic photosynthesis in purple bacteria,M00165,Reductive pentose phosphate cycle (Calvin cycle)
4737,Signature module,Gene set,Metabolic capacity,M00613,Anoxygenic photosynthesis in green nonsulfur b...,M00597,Anoxygenic photosystem II
4738,Signature module,Gene set,Metabolic capacity,M00613,Anoxygenic photosynthesis in green nonsulfur b...,M00376,3-Hydroxypropionate bi-cycle


In [13]:
clusterRes_df[clusterRes_df['dominant phylum'].str.contains('Micrarchaeo')]

,#CRISPRS,CRISPRS,NoCOG,assembly,chip,chip counts,cluster#,dominant domain,dominant phylum,dominant species,...,highest_lineage_name,length,majorCOG,mu_x,mu_y,num_contigs,number of cells,sample,sig_x,sig_y
238,0.0,NaN,NaN,mega,5.0,NaN,12.0,Archaea,Candidatus Micrarchaeota,unclassified,...,,268227.0,1.0,-24.559164,46.653121,40.0,5.270336,Obsidian6,3.893585,6.631651
271,0.0,NaN,NaN,meta,3.0,NaN,11.0,Archaea,Candidatus Micrarchaeota,unclassified,...,unclassified,2919304.0,0.0,-16.205998,46.712373,257.0,3.096265,Obsidian6,8.535439,4.909309


In [60]:
KEGGmoddf

,A,B,C,D-module,D,E-module,E
0,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00855,"PRK, prkB; phosphoribulokinase [EC:2.7.1.19]"
1,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01601,rbcL; ribulose-bisphosphate carboxylase large ...
2,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01602,rbcS; ribulose-bisphosphate carboxylase small ...
3,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00927,"PGK, pgk; phosphoglycerate kinase [EC:2.7.2.3]"
4,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K05298,GAPA; glyceraldehyde-3-phosphate dehydrogenase...
5,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00150,gap2; glyceraldehyde-3-phosphate dehydrogenase...
6,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00134,"GAPDH, gapA; glyceraldehyde 3-phosphate dehydr..."
7,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01623,"ALDO; fructose-bisphosphate aldolase, class I ..."
8,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01624,"FBA, fbaA; fructose-bisphosphate aldolase, cla..."
9,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K03841,"FBP, fbp; fructose-1,6-bisphosphatase I [EC:3...."
